In [269]:
import pygris
from pygris.data import get_census
from pygris import blocks, block_groups
from sodapy import Socrata
import pandas as pd
import os
import requests
from census import Census
from us import states
import config

In [267]:
# Grab Census Key & CDC App token from text file
# On the first line of keys.txt file put your census API key (obtained here: https://api.census.gov/data/key_signup.html)
# On the second line, place your CDC App token (obtained here: https://chronicdata.cdc.gov/login)
keyFile = open('keys.txt', 'r')
key = config.api_key
app_token = config.app_tkn
# Establish connection to CDC to extract BRFSS data
client = Socrata("data.cdc.gov", 
                 app_token, 
                 timeout=10)

# Prepare the Census library client with API key
c = Census(key)

# Create a dictionary with the 50 (plus DC) states and their respective code for census information
# State abbreviation: numeric code
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46'
}


# Dictionaries containing the income maximums to receive SNAP
# Income threshold changes every year (October of prior year to September of current year)
# year: [Income limit for single person household, limit increase per person]
income_dict = {
	2013: [1211, 429],
	2014: [1245, 436],
	2015: [1265, 440],
	2016: [1276, 451],
	2017: [1287, 451],
	2018: [1307, 453],
	2019: [1316, 468],
	2020: [1354, 479],
	2021: [1383, 486],
	2022: [1396, 492]
}

AK_income_dict = {
	2013: [1514, 537],
	2014: [1555, 545],
	2015: [1580, 551],
	2016: [1595, 564],
	2017: [1608, 564],
	2018: [1632, 567],
	2019: [1645, 585],
	2020: [1690, 600],
	2021: [1728, 607],
	2022: [1744, 616]
}

HI_income_dict = {
	2013: [1394, 493],
	2014: [1434, 501],
	2015: [1454, 506],
	2016: [1468, 518],
	2017: [1481, 518],
	2018: [1502, 522],
	2019: [1513, 539],
	2020: [1558, 551],
	2021: [1591, 558],
	2022: [1606, 566]
}

In [193]:
# Ready a list to hold queried data
data_hold = []

# Iterate through each state code
### Iterate through target years: 2018 to 2022
### Retrieve census data for the state that year to get SNAP usage data
for state in state_codes: 
    # Hold the state code in the format Pygris uses ("state:10" for Delaware)
    in_state = "state:" + str(state_codes[state])
    for year in range(2013, 2023): 
        # hse_data - tract level -> summed to state level
        # hhs_inc - block group level -> census collates to state level
        ## hhs_inc utilizes the Census library rather than the Pygris library due to execution time
        
        # Extract snap allocation for the State-Year from the census
        hse_data = get_census(dataset = "acs/acs5",
                                   variables = ["B99221_002E", "B99221_003E"],
                                   year = year,
                                   params = {
                                     "key": key,
                                     "for": "tract:*",
                                     "in": in_state},
                                   guess_dtypes = True,
                                   return_geoid = True
        )
        # Extract average household size and income levels for the State-Year
        # Census library call to retrieve the year's average household size
        hhs_inc = c.acs5.get(('NAME', 'B25010_001E', "B19001_001E", "B19001_002E","B19001_003E",
                              "B19001_004E", "B19001_005E", "B19001_006E","B19001_007E","B19001_008E"), 
                             {'for': 'state:{}'.format(state_codes[state])}, 
                             year=year)
        
        # Hawaii and Alaska have higher SNAP limits than other states + DC
        # Set the single person household limit (base_inc) and the per person limit addition (ppa) applicable values of State-Year
        if state == 'HI':
            base_inc = HI_income_dict[year][0]
            ppa = HI_income_dict[year][1]
        elif state == 'AK':
            base_inc = AK_income_dict[year][0]
            ppa = AK_income_dict[year][1]
        else:
            base_inc = income_dict[year][0]
            ppa = income_dict[year][1]
            
        # Calculate the average qualifying income limit 
        # Average state household size taken, minus one as base_inc accounts for 1 person
        avg_q_income = (base_inc+(ppa*(hhs_inc[0]['B25010_001E']-1)))*12

        data_hold.append([state, year, hse_data["B99221_002E"].sum(), hse_data["B99221_003E"].sum(), hhs_inc[0]['B25010_001E'], 
                        avg_q_income, hhs_inc[0]['B19001_001E'], hhs_inc[0]['B19001_002E'],
                        hhs_inc[0]['B19001_003E'], hhs_inc[0]['B19001_004E'], hhs_inc[0]['B19001_005E'],
                        hhs_inc[0]['B19001_006E'], hhs_inc[0]['B19001_007E'], hhs_inc[0]['B19001_008E']])
        #data_hold.append([state, year, hse_data["B99221_002E"].sum(), hse_data["B99221_003E"].sum()])

In [195]:
# Convert list to a DataFrame with proper column titles
df = pd.DataFrame(data_hold, columns = ['State', 'Year', 'SNAP_Allocated', 'SNAP_NA', 'Avg_HH_Size', 'Avg_Q_Income', 
                                        "Total_People", "HHI<10k", "10k<=HHI<15k", "15k<=HHI<20k", "20k<=HHI<25k", 
                                        "25k<=HHI<30k", "30k<=HHI<35k", "35k<=HHI<40k"])

In [227]:
# Retrieve CHC data from BRFSS and place into a DataFrame
CHC = client.get_all("dttw-5yxu", where="locationabbr NOT IN ('US','UW','VI','GU','PR') AND year BETWEEN 2013 AND 2022 AND class = 'Chronic Health Indicators' AND topic IN ('Depression', 'Cardiovascular Disease', 'Kidney', 'Diabetes') AND break_out_category IN ('Overall', 'Household Income')")
CHC_df = pd.DataFrame.from_records(CHC).astype("string")
CHC_df['year'] = CHC_df['year'].astype(int)

In [275]:
# Join census data to BRFSS data on State - Year
fuse_df = pd.merge(df, CHC_df,  how='outer', left_on=['State','Year'], right_on = ['locationabbr','year'])

In [277]:
# Drop columns with unneeded data
fuse_df = fuse_df.drop(['locationabbr','confidence_limit_low','confidence_limit_high','display_order','data_value_unit',
        'data_value_type','datasource','classid','topicid','locationid','breakoutid','breakoutcategoryid',
        'questionid','responseid','geolocation','data_value_footnote_symbol','data_value_footnote'], axis=1)

In [279]:
# Retype data to proper types, add in calculated columns
fuse_df.insert(4, 'Total', fuse_df['SNAP_Allocated']+ fuse_df['SNAP_NA'])
fuse_df.insert(5, 'Pct_Allo', (fuse_df['SNAP_Allocated'] / fuse_df['Total'])*100)
fuse_df[["State","locationdesc","class","topic","question","response","break_out","break_out_category"]] = fuse_df[["State","locationdesc","class","topic","question","response","break_out","break_out_category"]].astype("string")
fuse_df = fuse_df.dropna(axis=0, subset=['sample_size'])
fuse_df[["Total_People", "HHI<10k", "10k<=HHI<15k","15k<=HHI<20k","20k<=HHI<25k","25k<=HHI<30k","30k<=HHI<35k","35k<=HHI<40k",'sample_size']] = fuse_df[["Total_People", "HHI<10k", "10k<=HHI<15k","15k<=HHI<20k","20k<=HHI<25k","25k<=HHI<30k","30k<=HHI<35k","35k<=HHI<40k",'sample_size']].astype(int) # Total_People Should be identical to Total column, left as a verification tool
fuse_df.insert(16, 'HHI>=40k', (fuse_df["Total_People"]-(fuse_df["HHI<10k"]+fuse_df["10k<=HHI<15k"]+fuse_df["15k<=HHI<20k"]+fuse_df["20k<=HHI<25k"]+fuse_df["25k<=HHI<30k"]+fuse_df["30k<=HHI<35k"]+fuse_df["35k<=HHI<40k"])))

In [30]:
# Export data to CSV
fuse_df.to_csv('hold/Manual_10yr_SNAP_7-11.csv', index=False) 